# Basic Logistic Regression Model to Predict NFL Outcomes
This code walks you through how to build and use a machine learning model to predict NFL games. Specifically, we are using a logistic regression classification model. In this example, we are simply predicting the probability of wins and losses, but I am developing a separate code to simultaneously predict wins/losses and whether teams will cover the spread, hit the over/under, etc.

## Step 1: Import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, brier_score_loss, classification_report

from collections import defaultdict
import kagglehub
from kagglehub import KaggleDatasetAdapter

C:\Users\Nathan Cristello\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2: Data Management

In [2]:
# ---- Team abbreviations ----
team_map = {
    "ARI": "Arizona Cardinals",
    "ATL": "Atlanta Falcons",
    "BAL": "Baltimore Ravens",
    "BUF": "Buffalo Bills",
    "CAR": "Carolina Panthers",
    "CHI": "Chicago Bears",
    "CIN": "Cincinnati Bengals",
    "CLE": "Cleveland Browns",
    "DAL": "Dallas Cowboys",
    "DEN": "Denver Broncos",
    "DET": "Detroit Lions",
    "GB": "Green Bay Packers",
    "HOU": "Houston Texans",
    "IND": "Indianapolis Colts",
    "JAX": "Jacksonville Jaguars",
    "KC": "Kansas City Chiefs",
    "LV": "Las Vegas Raiders",
    "LAC": "Los Angeles Chargers",
    "LAR": "Los Angeles Rams",
    "MIA": "Miami Dolphins",
    "MIN": "Minnesota Vikings",
    "NE": "New England Patriots",
    "NO": "New Orleans Saints",
    "NYG": "New York Giants",
    "NYJ": "New York Jets",
    "PHI": "Philadelphia Eagles",
    "PIT": "Pittsburgh Steelers",
    "SEA": "Seattle Seahawks",
    "SF": "San Francisco 49ers",
    "TB": "Tampa Bay Buccaneers",
    "TEN": "Tennessee Titans",
    "WAS": "Washington Commanders"
}

team_map_flipped = {v: k for k, v in team_map.items()}

In [3]:
# ---- Add "momentum" factor with this function ----
def add_team_form_features(games: pd.DataFrame, window: int = 5) -> pd.DataFrame:
    """
    For each game, create: 
    - home_streak, away_streak: win/loss streak going into the game
    - home/away_avg_points_for: average points scored over last 'window' games
    - home/away_avg_points_against: average points allows over last 'window' games
    All computed using only past games
    """
    games = games.sort_values('schedule_date')

    state = defaultdict(lambda: {
        "streak": 0,
        "recent_for": [],
        "recent_against": []
    })

    home_streak, away_streak, home_avg_for, home_avg_against, away_avg_for, away_avg_against = [], [], [], [], [], []
    for _, row in games.iterrows():
        home = row['team_home']
        away = row['team_away']
        sh = row['score_home']
        sa = row['score_away']
        home_won = sh > sa

        h_state = state[home]
        a_state = state[away]

        home_streak.append(h_state['streak'])
        away_streak.append(a_state['streak'])

        home_avg_for.append(np.median(h_state['recent_for']) if h_state['recent_for'] else 0.0)
        home_avg_against.append(np.median(h_state['recent_against'] if h_state['recent_against'] else 0.0))
        away_avg_for.append(np.median(a_state['recent_for']) if a_state['recent_for'] else 0.0)
        away_avg_against.append(np.median(a_state['recent_against'] if a_state['recent_against'] else 0.0))

        h_state['recent_for'].append(sh)
        h_state['recent_against'].append(sa)
        h_state['streak'] = h_state['streak'] + 1 if home_won else -1

        a_state['recent_for'].append(sa)
        a_state['recent_against'].append(sh)
        a_state['streak'] = a_state['streak'] + 1 if home_won else -1

        if len(h_state['recent_for']) > window:
            h_state['recent_for'].pop(0)
            h_state['recent_against'].pop(0)
        if len(a_state['recent_for']) > window:
            a_state['recent_for'].pop(0)
            a_state['recent_against'].pop(0)

    games['home_streak'] = home_streak
    games['away_streak'] = away_streak
    games['home_avg_pts_for'] = home_avg_for
    games['home_avg_pts_against'] = home_avg_against
    games['away_avg_pts_for'] = away_avg_for
    games['away_avg_pts_against'] = away_avg_against

    return games

In [4]:
# ---- Data loading/improvements/additions ----

# Read in data, loading in the latest version (is updated weekly)
nfl_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "tobycrabtree/nfl-scores-and-betting-data",
  "spreadspoke_scores.csv"
)

nfl_df['schedule_date'] = pd.to_datetime(nfl_df['schedule_date'])

# Keeping (non-playoff) games played during and after the 2015 season.
date_sel = ((nfl_df['schedule_season'] >= 2015) 
            & (nfl_df['schedule_date'] < pd.to_datetime('today'))
            & (nfl_df['schedule_playoff'] == False))
nfl_df = nfl_df[date_sel]

# Adding columns to find over/under hits
nfl_df['true_total'] = nfl_df['score_home'] + nfl_df['score_away']
nfl_df['over_hit'] = np.where(nfl_df['true_total'] > nfl_df['over_under_line'].astype(float), True, False)
nfl_df['under_hit'] = np.where(nfl_df['true_total'] < nfl_df['over_under_line'].astype(float), True, False)
nfl_df['push_hit'] = np.where(nfl_df['true_total'] == nfl_df['over_under_line'].astype(float), True, False)

# Flags for winning team and home/away wins
nfl_df['home_win'] = np.where(nfl_df['score_home'] > nfl_df['score_away'], True, False)
nfl_df['away_win'] = np.where(nfl_df['score_away'] > nfl_df['score_home'], True, False)
nfl_df['tie_game'] = np.where(nfl_df['score_away'] == nfl_df['score_home'], True, False)
nfl_df['winning_team'] = np.where(nfl_df['score_home'] > nfl_df['score_away'], nfl_df['team_home'], nfl_df['team_away'])
nfl_df['winning_team_id'] = nfl_df['winning_team'].map(team_map_flipped)

# Flags for finding if the spread was covered
nfl_df['favorite_score'] = np.where(nfl_df['team_favorite_id'] == nfl_df['team_home'], nfl_df['score_home'], nfl_df['score_away'])
nfl_df['underdog_score'] = np.where(nfl_df['team_favorite_id'] == nfl_df['team_home'], nfl_df['score_away'], nfl_df['score_home'])
nfl_df['spread_margin'] = nfl_df['favorite_score'] - nfl_df['underdog_score']
nfl_df['spread_result'] = np.where(nfl_df['spread_margin'] > np.abs(nfl_df['spread_favorite']), 1,
                                   np.where(nfl_df['spread_margin'] == np.abs(nfl_df['spread_favorite']), 0.5, 0))

# Sort the data
nfl_df = nfl_df.sort_values(by=['schedule_date', 'schedule_season', 'schedule_week']).reset_index(drop=True)

# Drop scores we don't have
nfl_df = nfl_df.dropna(subset=['score_home', 'score_away'])

# Add in "momentum" over most recent 5 games
nfl_df = add_team_form_features(nfl_df, window=5)

C:\Users\Nathan Cristello\AppData\Local\Temp\ipykernel_11276\2749710627.py:4: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  nfl_df = kagglehub.load_dataset(


## Step 3: Model setup and training

In [5]:
# ---- Selecting columns to use in model ---- 
select_columns = [
    'schedule_season', 
    'schedule_week', 
    'schedule_playoff',
    'stadium_neutral',
    'home_streak',
    'away_streak',
    'home_avg_pts_for',
    'away_avg_pts_for',
    'home_avg_pts_against',
    'away_avg_pts_against',
    'spread_favorite',
    'over_under_line',
    'team_home',
    'team_away'
]

In [6]:
# ---- Create model dataframe ----
model_df = nfl_df[select_columns + ['home_win', 'over_hit', 'under_hit', 'push_hit', 'spread_result']].dropna().reset_index(drop=True)

X = model_df.drop(columns=['home_win'], axis=1)
y = model_df['home_win']

categorical_columns = ['schedule_week', 'team_home', 'team_away']
numeric_columns = [c for c in select_columns if c not in categorical_columns]

In [7]:
# ---- Time-based train/test split (training on games prior to 2025 season, keeping 2025 season as test data) ---- 
cut_season = 2024
train_mask = X['schedule_season'] <= cut_season

X_train = X[train_mask]
y_train = y[train_mask]
X_test = X[~train_mask]
y_test = y[~train_mask]

In [8]:
# ---- Build pipeline: preprocess + logistic regression ----
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocess = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ]
)

clf = LogisticRegression(max_iter=1000)

model = Pipeline(
    steps = [
        ('preprocess', preprocess),
        ('clf', clf)
    ]
)

In [9]:
# ---- Train and evaluate ----
model.fit(X, y)

y_prob = model.predict_proba(X_test)[:, 1] # probability that home team wins
y_pred = (y_prob > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))
print("Brier score:", brier_score_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))

Accuracy: 0.671875
ROC AUC: 0.749938830437974
Brier score: 0.20273413628908143
              precision    recall  f1-score   support

       False       0.65      0.69      0.67       122
        True       0.70      0.66      0.68       134

    accuracy                           0.67       256
   macro avg       0.67      0.67      0.67       256
weighted avg       0.67      0.67      0.67       256



## Step 4: Backtest model for 2025 season
Here, we'll evaluate how our model does in 2025 (classification report above shows that model is correct 67% of the time for the 2025 season). Let's verify this and check how it does on a team-by-team basis.

In [13]:
szn_2025 = nfl_df[nfl_df['schedule_season'] == 2025]

In [31]:
# ---- For loop to iterate through each game of 2025 and count the number of times the model correctly predicts games ----
model_right = 0
model_wrong = 0

team_correct = defaultdict(int) # number of times model is correct for every team
team_total = defaultdict(int)   # total games played per team (just to assert each team played 16 games)

for i in range(len(szn_2025)):
    this_game = szn_2025.iloc[i]

    mask = (
        (nfl_df['team_home'] == this_game['team_home']) &
        (nfl_df['team_away'] == this_game['team_away'])
    )
    eg_cow_games = nfl_df[mask].sort_values('schedule_date')

    if eg_cow_games.empty:
        continue

    last_meeting = eg_cow_games.iloc[-1]
    x_example = last_meeting[select_columns].to_frame().T
    prob_team_home_win = model.predict_proba(x_example)[0, 1]

    # Model prediction
    model_home_win = prob_team_home_win > 0.50
    actual_home_win = this_game['home_win']

    correct = (model_home_win == actual_home_win)

    # Prediction counts
    if correct:
        model_right += 1
    else:
        model_wrong += 1

    # Update per-team stats 
    home = this_game['team_home']
    away = this_game['team_away']

    team_total[home] += 1
    team_total[away] += 1

    if correct:
        team_correct[home] += 1
        team_correct[away] += 1

print("In 2025, this model got " + str(model_right) + " games correct and " + str(model_wrong) + " games wrong.")

In 2025, this model got 172 games correct and 84 games wrong.


In [34]:
team_accuracy = {
    team: team_correct[team] / team_total[team]
    for team in team_total
}

best_teams = sorted(team_accuracy.items(), key=lambda x: x[1], reverse=True)
worst_teams = sorted(team_accuracy.items(), key=lambda x: x[1])

print("Best teams for the model:")
for team, acc in best_teams[:5]:
    print(f"{team}: {acc:.3f}")

print("\nWorst teams for the model:")
for team, acc in worst_teams[:5]:
    print(f"{team}: {acc:.3f}")

Best teams for the model:
New York Giants: 0.875
Seattle Seahawks: 0.875
New York Jets: 0.875
New Orleans Saints: 0.875
New England Patriots: 0.875

Worst teams for the model:
Kansas City Chiefs: 0.375
Atlanta Falcons: 0.438
Dallas Cowboys: 0.500
Los Angeles Chargers: 0.500
Carolina Panthers: 0.500


In [37]:
def print_dictionary(dct):
    print("Number of games correctly predicted per team:")
    for team, num_games in dct.items():
        print("{}: {}".format(team, num_games))

print_dictionary(team_correct)

Number of games correctly predicted per team:
Philadelphia Eagles: 10
Dallas Cowboys: 8
Los Angeles Rams: 12
Houston Texans: 11
Washington Commanders: 13
New York Giants: 14
Seattle Seahawks: 14
San Francisco 49ers: 10
New York Jets: 14
Pittsburgh Steelers: 9
New Orleans Saints: 14
Arizona Cardinals: 13
Jacksonville Jaguars: 11
Carolina Panthers: 8
New England Patriots: 14
Las Vegas Raiders: 13
Green Bay Packers: 9
Detroit Lions: 12
Denver Broncos: 8
Tennessee Titans: 13
Cleveland Browns: 12
Cincinnati Bengals: 12
Atlanta Falcons: 7
Tampa Bay Buccaneers: 11
Chicago Bears: 9
Minnesota Vikings: 8
Buffalo Bills: 10
Kansas City Chiefs: 6
Baltimore Ravens: 8
Los Angeles Chargers: 8
Miami Dolphins: 10
Indianapolis Colts: 13
